In [2]:
import torch

In [3]:
from datasets import load_dataset

In [4]:
import pandas as pd

In [5]:
from textwrap import wrap
import matplotlib.pyplot as plt
import numpy as np

In [6]:
import os
import datasets
from transformers import VisionEncoderDecoderModel, AutoFeatureExtractor,AutoTokenizer

2023-06-18 19:59:09.267115: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
import nltk
try:
    nltk.data.find("tokenizers/punkt")
except (LookupError, OSError):
    nltk.download("punkt", quiet=True)

In [8]:
model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

Some weights of the model checkpoint at nlpconnect/vit-gpt2-image-captioning were not used when initializing VisionEncoderDecoderModel: ['decoder.transformer.h.0.crossattention.bias', 'decoder.transformer.h.10.crossattention.masked_bias', 'decoder.transformer.h.3.attn.bias', 'decoder.transformer.h.5.crossattention.masked_bias', 'decoder.transformer.h.7.crossattention.masked_bias', 'decoder.transformer.h.9.attn.masked_bias', 'decoder.transformer.h.2.attn.masked_bias', 'decoder.transformer.h.1.attn.masked_bias', 'decoder.transformer.h.6.attn.masked_bias', 'decoder.transformer.h.11.crossattention.masked_bias', 'decoder.transformer.h.3.crossattention.masked_bias', 'decoder.transformer.h.5.attn.bias', 'decoder.transformer.h.10.attn.masked_bias', 'decoder.transformer.h.0.crossattention.masked_bias', 'decoder.transformer.h.6.attn.bias', 'decoder.transformer.h.2.attn.bias', 'decoder.transformer.h.3.attn.masked_bias', 'decoder.transformer.h.4.attn.masked_bias', 'decoder.transformer.h.1.crossatt

In [9]:
from transformers import ViTImageProcessor, AutoTokenizer
feature_extractor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")


In [10]:
output_dir = "vit-gpt-model"
model.save_pretrained(output_dir)
feature_extractor.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

('vit-gpt-model/tokenizer_config.json',
 'vit-gpt-model/special_tokens_map.json',
 'vit-gpt-model/vocab.json',
 'vit-gpt-model/merges.txt',
 'vit-gpt-model/added_tokens.json',
 'vit-gpt-model/tokenizer.json')

In [12]:
ds = load_dataset("imagefolder", data_dir="../dataset/my_dataset/train")['train'].train_test_split(test_size=0.2035)

Resolving data files:   0%|          | 0/1257 [00:00<?, ?it/s]

Found cached dataset imagefolder (/home1/sm21mtech14004/.cache/huggingface/datasets/imagefolder/default-f50bdc978128de85/0.0.0/37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f)


  0%|          | 0/1 [00:00<?, ?it/s]

In [20]:
for i in (ds['test']):
    print(i)

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=4608x2176 at 0x7F480FD9D150>, 'text': 'an aerial view of many motorcycles and bicycles are parked on the sidewalk'}
{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=2688x1512 at 0x7F480E7602B0>, 'text': 'a tennis court with a large metal structure in the background'}
{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1536x827 at 0x7F480E760610>, 'text': 'an aerial view of a road with a tower'}
{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=2688x1512 at 0x7F480E7607C0>, 'text': 'an aerial view of a road with a triangle shaped building in the middle of park'}
{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1070x649 at 0x7F480E760A00>, 'text': 'red and white coloured building roundabout  ahead of it and green bushes on left side of it'}
{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1127x759 at 0x7F480E7604C0>, 'text': 'a road divided by green

In [13]:
from PIL import Image

# text preprocessing step
def tokenization_fn(captions, max_target_length):
    """Run tokenization on captions."""
    labels = tokenizer(captions, 
                      padding="max_length", 
                      max_length=max_target_length).input_ids

    return labels

In [14]:
def feature_extraction_fn(image_paths, check_image=True):
    """
    Run feature extraction on images
    If `check_image` is `True`, the examples that fails during `Image.open()` will be caught and discarded.
    Otherwise, an exception will be thrown.
    """

    model_inputs = {}

    if check_image:
        images = []
        to_keep = []
        for image_file in image_paths:
            try:
                img = image_file#image#Image.open(image_file)
                images.append(img)
                to_keep.append(True)
            except Exception:
                to_keep.append(False)
    else:
        images = [image_file for image_file in image_paths]

    encoder_inputs = feature_extractor(images=images, return_tensors="np")

    return encoder_inputs.pixel_values

def preprocess_fn(examples, max_target_length, check_image = True):
    """Run tokenization + image feature extraction"""
    image_paths = examples['image']
    captions = examples['text']    
    
    model_inputs = {}
    # This contains image path column
    model_inputs['labels'] = tokenization_fn(captions, max_target_length)
    model_inputs['pixel_values'] = feature_extraction_fn(image_paths, check_image=check_image)

    return model_inputs

In [15]:
processed_dataset = ds.map(
    function=preprocess_fn,
    batched=True,
    fn_kwargs={"max_target_length": 128},
    remove_columns=ds['train'].column_names
)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/256 [00:00<?, ? examples/s]

In [14]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    evaluation_strategy="epoch",
    num_train_epochs=5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    output_dir="./image-captioning-output",
    save_strategy="epoch"
)

In [15]:
import evaluate
metric = evaluate.load("rouge")

In [16]:
!pip install rouge_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [17]:
import numpy as np

ignore_pad_token_for_loss = True


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    if ignore_pad_token_for_loss:
        # Replace -100 in the labels as we can't decode them.
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds,
                                                     decoded_labels)

    result = metric.compute(predictions=decoded_preds,
                            references=decoded_labels,
                            use_stemmer=True)
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds
    ]
    result["gen_len"] = np.mean(prediction_lens)
    return result

In [18]:
from transformers import default_data_collator

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=feature_extractor,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=processed_dataset['train'],
    eval_dataset=processed_dataset['test'],
    data_collator=default_data_collator,
)

In [19]:
trainer.train()

/home1/sm21mtech14004/anaconda3/envs/my_env/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/home1/sm21mtech14004/anaconda3/envs/my_env/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,0.231237,38.152700,13.242200,31.224700,31.205900,11.992188
2,No log,0.206117,40.550800,16.159600,34.018800,34.011800,10.289062
3,No log,0.197625,41.150600,16.203300,33.502800,33.553400,11.187500
4,No log,0.195211,41.568600,17.718200,34.448700,34.372300,10.960938
5,No log,0.194698,41.496400,17.630400,34.663900,34.616800,11.308594


/home1/sm21mtech14004/anaconda3/envs/my_env/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home1/sm21mtech14004/anaconda3/envs/my_env/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home1/sm21mtech14004/anaconda3/envs/my_env/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home1/sm21mtech14004/anaconda3/envs/my_env/lib/python3.10/site-packages/torch/

TrainOutput(global_step=160, training_loss=0.23717291355133058, metrics={'train_runtime': 602.5153, 'train_samples_per_second': 8.299, 'train_steps_per_second': 0.266, 'total_flos': 9.0231945560064e+17, 'train_loss': 0.23717291355133058, 'epoch': 5.0})